In [1]:
%matplotlib inline

In [34]:
from pathlib import Path
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot
import eelbrain
import mne
import trftools

from pprint import pprint
import numpy as np
from scipy import stats

In [24]:
# Load in the VST's score in ESLs

VST_Score_STR_LIST = ['6.7', '7.3', '7.8', '8.2', '8.4', '6.4', '7.5', '6.7', '5.2', '5.3', '6.5'
                 , '5.1', '6.1', '7.9', '8.7', '8.0', '8.8', '6.4', '7.0', '7.4', '6.6', '7.2'
                 , '7.0', '7.3', '7.3', '7.7']  # 26 subs
VST_Score_float_LIST = [6.7, 7.3, 7.8, 8.2, 8.4, 6.4, 7.5, 6.7
                        , 5.2, 5.3, 6.5, 5.1, 6.1, 7.9, 8.7, 8.0
                        , 8.8, 6.4, 7.0, 7.4, 6.6, 7.2, 7.0, 7.3, 7.3, 7.7]

# exclude sub: 14 / 18 / 33 / 37
sub_idLIST = [10, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27
              , 28, 29, 30, 31, 32, 34, 35, 36, 38, 39]
# Female = 1; Male = 2
sub_SexLIST = [ 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

ds = Dataset()
ds['sub_id'] = Var(np.asarray(VST_Score_float_LIST))
ds['VST_score'] = Var(np.asarray(VST_ScoreLIST))
ds['Sex'] = Var(np.asarray(sub_SexLIST))

print(ds)
'''
# initialize an empty Dataset:
ds = Dataset()
# numeric values are added as Var object:
ds['y'] = Var(numpy.random.normal(0, 1, 6))
# categorical data as represented in Factors:
ds['a'] = Factor(['a', 'b', 'c'], repeat=2)
# A variable that's equal in all cases can be assigned quickly:
ds[:, 'z'] = 0.
# check the result:
ds
'''

TypeError: x of type unicode (dtype dtype('<U3')): Var needs numerical data type. Consider using a Datalist instead.

In [25]:
VST_Score_STR_LIST = ['6.7', '7.3', '7.8', '8.2', '8.4', '6.4', '7.5', '6.7', '5.2', '5.3', '6.5'
                 , '5.1', '6.1', '7.9', '8.7', '8.0', '8.8', '6.4', '7.0', '7.4', '6.6', '7.2'
                 , '7.0', '7.3', '7.3', '7.7']  # 26 subs

tmpLIST = []
for i in VST_Score_STR_LIST:
    tmp = float(i)
    tmpLIST.append(tmp)
    
print(tmpLIST)

[6.7, 7.3, 7.8, 8.2, 8.4, 6.4, 7.5, 6.7, 5.2, 5.3, 6.5, 5.1, 6.1, 7.9, 8.7, 8.0, 8.8, 6.4, 7.0, 7.4, 6.6, 7.2, 7.0, 7.3, 7.3, 7.7]


In [19]:
p = plot.Scatter(str(ds['VST_score']), 'Sex', data=ds, h=3)

KeyError: 'VST_score'

In [7]:
VST_ScoreLIST = np.asarray([6.7, 7.3, 7.8, 8.2, 8.4, 6.4, 7.5, 6.7, 5.2, 5.3, 6.5, 5.1, 6.1
                 , 7.9, 8.7, 8, 8.8, 6.4, 7, 7.4, 6.6, 7.2, 7, 7.3, 7.3, 7.7])  # 26 subs
print(len(VST_ScoreLIST))
print(type(VST_ScoreLIST))

26
<class 'numpy.ndarray'>


In [3]:
## ESLs ##

#STIMULI = [str(i) for i in range(1, 13)]
DATA_ROOT = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")  #Path("~").expanduser() / 'Data' / 'Alice'
#PREDICTOR_audio_DIR = DATA_ROOT / 'TRFs_pridictors/audio_predictors'
#PREDICTOR_word_DIR = DATA_ROOT / 'TRFs_pridictors/word_predictors'
EEG_DIR = DATA_ROOT / 'EEG_ESLs' / 'Alice_ESL_ICAed_fif'
ESL_SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'n_2_S\d*', path.name)]  #S01_alice-raw.fif
# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs_ESLs'
TRF_DIR.mkdir(exist_ok=True)
print(ESL_SUBJECTS)
print(len(ESL_SUBJECTS))  # 26

DST = TRF_DIR / 'ESLs_figures'
DST.mkdir(exist_ok=True)

['n_2_S019_ICAed_raw.fif', 'n_2_S020_ICAed_raw.fif', 'n_2_S021_ICAed_raw.fif', 'n_2_S022_ICAed_raw.fif', 'n_2_S023_ICAed_raw.fif', 'n_2_S026_ICAed_raw.fif', 'n_2_S024_ICAed_raw.fif', 'n_2_S012_ICAed_raw.fif', 'n_2_S013_ICAed_raw.fif', 'n_2_S015_ICAed_raw.fif', 'n_2_S016_ICAed_raw.fif', 'n_2_S011_ICAed_raw.fif', 'n_2_S010_ICAed_raw.fif', 'n_2_S029_ICAed_raw.fif', 'n_2_S027_ICAed_raw.fif', 'n_2_S028_ICAed_raw.fif', 'n_2_S030_ICAed_raw.fif', 'n_2_S031_ICAed_raw.fif', 'n_2_S017_ICAed_raw.fif', 'n_2_S025_ICAed_raw.fif', 'n_2_S032_ICAed_raw.fif', 'n_2_S034_ICAed_raw.fif', 'n_2_S035_ICAed_raw.fif', 'n_2_S036_ICAed_raw.fif', 'n_2_S038_ICAed_raw.fif', 'n_2_S039_ICAed_raw.fif']
26


In [11]:
### For ESLs  ###
#subj = 2
"""
subject_nSTR_p = "n_S%.3d_ICAed_raw.fif" %subj

raw = mne.io.read_raw_fif(EEG_DIR / f'{subject_nSTR_p}', preload=True)

montage_x = eelbrain.load.fiff.sensor_dim(raw.info)
#temp_data = eeg_.T *1e+6
eeg_concatenated = eelbrain.NDVar(temp_data, (time, montage_x), name='EEG', info={'unit': 'µV'})

SUBJECTS = ['1', '2', '3', '4', '5', '6', '7']
rows = []
for subject in SUBJECTS:
    trf = eelbrain.load.unpickle('Subj%s_TRFs_envelop_w_on.pickle'%(subject))
    rows.append([subject, trf.proportion_explained, trf.h[0].abs()])
    pass
data_env = eelbrain.Dataset.from_caselist(['subject', 'det', 'trf'], rows)
"""
#subj_sLIST = [3, 4, 5, 6, 7, 8]
subj_sLIST = []
for subj_name in ESL_SUBJECTS:
    subj_sLIST.append(int(subj_name[5:8]))
rowsLIST = []

# res.h[0] >> <NDVar 'gammatone': 57 sensor, 8 frequency, 114 time>

## TRFs Envelope  ##
for subj in  subj_sLIST:
    res = eelbrain.load.unpickle(TRF_DIR / Path('S%.3d/S%.3d envelope.pickle'%(subj, subj)))
    rowsLIST.append([subj, res.proportion_explained, res.h[0]])#.abs()])
data_env = eelbrain.Dataset.from_caselist(['subject', 'det', 'trf'], rowsLIST)
#pprint(rowsLIST)
#print(data_env)


#test that model predictive power on held-out data is > 0
test_envelope = eelbrain.testnd.TTestOneSample('det', ds=data_env, tail=1, pmin=0.05)
#p = eelbrain.plot.Topomap(test_envelope)
#cb = p.plot_colorbar(width=0.1)


# ## Envelope TRF
# Test the TRF with a one-sample *t*-test against 0. This tests the null-hypothesis that the electrical current direction at each time point was random across subjects. The systematic current directions shown below at anterior electrodes are typical of auditory responses. 

trf_envelope = eelbrain.testnd.TTestOneSample('trf', ds=data_env, pmin=0.05)
print(trf_envelope)  # print out the stats results
onesample_clus = trf_envelope.find_clusters(0.05)
print(onesample_clus) # print out the cluster result

# gammatone >> ,t=[0.87]
# gammatone_onset >> ,t=[0.42, 0.52]
#p1 = eelbrain.plot.TopoArray(trf_envelope, t=[0.13, 0.35, 0.66, 0.99], clip='circle', cmap='xpolar', h=4, w = 8) #, t=[0.08, 0.16, 0.22, 0.45]
#cb = p.plot_colorbar(width=0.1)

#p1.save(DST / 'ESLs_ALL26_Envelope TRF_envelop.png')

#x = trf的結果

<TTestOneSample 'trf', samples=10000, pmin=0.05, 32 clusters, p < .001>
id    n_sensors   tstart   tstop   duration   v         p        sig
--------------------------------------------------------------------
1     51          -0.12    0.45    0.57       3995.8    0        ***
265   36          0.56     0.82    0.26       -612.63   0.0176   *  
268   40          0.81     1.02    0.21       -968.33   0.0063   ** 


In [44]:
## TRF: envelope+onset ##
#subj_sLIST = [3, 4, 5, 6, 7, 8]

VST_Score_float_LIST = [6.7, 7.3, 7.8, 8.2, 8.4, 6.4, 7.5, 6.7
                        , 5.2, 5.3, 6.5, 5.1, 6.1, 7.9, 8.7, 8.0
                        , 8.8, 6.4, 7.0, 7.4, 6.6, 7.2, 7.0, 7.3, 7.3, 7.7]

rowsLIST = []
ESL_trf_valueLIST = []
for subj in  subj_sLIST:
    res = eelbrain.load.unpickle(TRF_DIR / Path('S%.3d/S%.3d envelope+onset.pickle'%(subj, subj)))
    rowsLIST.append([subj, res.proportion_explained, *res.h])#.abs()])
    print(res.h[0].mean())
    ESL_trf_value = res.h[0].mean()
    ESL_trf_valueLIST.append(ESL_trf_value)
    x_names = res.x
data_onset = eelbrain.Dataset.from_caselist(['subject', 'det', *x_names], rowsLIST)
print(len(data_onset['det']))
#pprint(rowsLIST)
#print(data_onset)

test_onset = eelbrain.testnd.TTestOneSample('det', ds=data_onset, tail=1, pmin=0.05)
# Paired t-test by specifying two measurement NDVars with matched cases
# Note that this presupposes that subjects are in the same order
test_onset_envelope = eelbrain.testnd.TTestRelated(data_onset['det'], data_env['det'], tail=1, pmin=0.05)

test_onsetEnv_VST = eelbrain.testnd.Correlation(data_onset['det'], VST_Score_float_LIST, pmin=0.05) #, VST_Score_float_LIST
print(test_onsetEnv_VST)

"""
p2 = eelbrain.plot.Topomap(
    [test_onset.masked_difference(), test_onset_envelope.masked_difference()], 
    axtitle=[['Envelope + Onsets\n', test_onset], ['Envelope + Onsets > Envelope\n', test_onset_envelope]],
    ncol=2, clip='circle')
cb = p.plot_colorbar(width=0.1)

p2.save(TRF_DIR / 'ESLs_ALL26_Envelope+onset-TRFs_topo.png')
"""

6.838001436339386e-05
-5.4747614650661744e-05
0.00016160100543757052
3.2817790089258233e-05
-1.3593926336308605e-06
-3.748589309531138e-05
-0.00010519903560069765
8.72704421873397e-05
0.0001268338976095209
0.0001623319995896173
3.707551041346056e-05
0.00021042372011901097
6.400687390992101e-05
2.2263260490407297e-05
0.00012121678465168771
6.754642454088437e-05
6.593054273109676e-05
0.0001350030778701139
0.00022542833692418182
-5.110546834923566e-05
-0.00015742023186621524
0.00013151482507438184
0.00024811480455524774
0.00012920642248897096
-1.2016517902944505e-05
-6.825176977531548e-05
26
<Correlation 'det', None, samples=10000, pmin=0.05, no clusters>


"\np2 = eelbrain.plot.Topomap(\n    [test_onset.masked_difference(), test_onset_envelope.masked_difference()], \n    axtitle=[['Envelope + Onsets\n', test_onset], ['Envelope + Onsets > Envelope\n', test_onset_envelope]],\n    ncol=2, clip='circle')\ncb = p.plot_colorbar(width=0.1)\n\np2.save(TRF_DIR / 'ESLs_ALL26_Envelope+onset-TRFs_topo.png')\n"

In [18]:
temp_h0 = res.h[0]
dir(temp_h0)

['__abs__',
 '__add__',
 '__and__',
 '__array_interface__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',


In [31]:
print(temp_h0.get_data().mean())

-6.825176977531548e-05


In [41]:
print(ESL_trf_valueLIST)

[array([[ 0.00000000e+00,  0.00000000e+00, -2.89120579e-20, ...,
         9.03501810e-21,  2.89120579e-20, -1.44560290e-20],
       [ 0.00000000e+00,  7.22801448e-20, -1.19047619e-05, ...,
         1.22247024e-03,  2.26190476e-04,  1.19047619e-05],
       [ 1.66666667e-04,  1.20833333e-03,  2.64583333e-03, ...,
         5.42101086e-20, -7.22801448e-21,  0.00000000e+00],
       ...,
       [ 5.35714286e-05,  3.61607143e-04,  6.69642857e-04, ...,
        -2.89120579e-20, -2.18469809e-20,  3.61400724e-20],
       [-2.89120579e-20, -2.89120579e-20, -1.44560290e-20, ...,
        -4.33680869e-20,  7.95081593e-20,  8.04116611e-20],
       [-5.78241159e-20,  0.00000000e+00, -2.89120579e-20, ...,
        -5.05961014e-20, -2.25875453e-20,  5.60171122e-20]]), array([[-8.67361738e-20,  5.78241159e-20, -1.66244333e-19, ...,
        -2.33657351e-19,  2.39977381e-20,  1.49402982e-19],
       [-5.78241159e-20,  8.67361738e-20,  7.22801448e-20, ...,
         1.13491593e-19,  3.59055394e-20,  5.47538905

In [45]:
VST_Array = np.array(VST_Score_float_LIST)
print(len(VST_Array))
ESL_trf_value_Array = np.array(ESL_trf_valueLIST)
print(len(ESL_trf_value_Array))

# Conducting pair t-test bwtween HCD & LCD evoked  #default= 雙尾檢定 >>這裡選"less" for單尾檢定 >> 因為是H-L，所以H放前面
corr_VST_OnsetEnv = stats.pearsonr(ESL_trf_value_Array, VST_Array)#, alternative='less')  
print(corr_VST_OnsetEnv)
print(type(corr_VST_OnsetEnv))


26
26
PearsonRResult(statistic=-0.2611446822340303, pvalue=0.19754025060083846)
<class 'scipy.stats._stats_py.PearsonRResult'>
